In [19]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
def connect_to_database():
    """Function for connecting to the SQL database"""

    # Create variables to store cretentials
    USER = "admin"
    DB = "dbikes"
    PORT = "3306"
    URL = "database-1.ctesjcult8dm.eu-west-1.rds.amazonaws.com"

    # Read in password from text file
    with open('../mysql_password.txt') as f:
        PASSWORD = ''.join(f.readlines())
        PASSWORD = str(PASSWORD).split()[0]

    # Create engine using credentials
    engine = create_engine(
        "mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)

    # Create connection using engine and return connection
    conn = engine.connect()
    return conn


In [3]:
engine = connect_to_database()

2022-03-23 13:38:59,874 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-03-23 13:38:59,876 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-23 13:39:00,109 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-03-23 13:39:00,112 INFO sqlalchemy.engine.Engine [generated in 0.00687s] {}
2022-03-23 13:39:00,204 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-23 13:39:00,206 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
availability_df = pd.read_sql_query("SELECT * FROM dbikes.availability", engine)
availability_df.head()

2022-03-23 13:39:00,460 INFO sqlalchemy.engine.Engine SELECT * FROM dbikes.availability
2022-03-23 13:39:00,464 INFO sqlalchemy.engine.Engine [raw sql] {}


,number,available_bikes,available_stands,last_update
0,42,17,13,1644921618000
1,30,15,5,1644922002000
2,54,16,17,1644922012000
3,108,10,25,1644921926000
4,56,9,31,1644922071000


In [5]:
availability_df.shape

(2825680, 4)

In [6]:
# Remove duplicates
availability_df.drop_duplicates(inplace=True)

In [7]:
print(availability_df.shape)
# availability_df.duplicated().sum()

(717692, 4)


In [8]:
weather_df = pd.read_sql_query("SELECT dt, temperature, main FROM dbikes.real_time_weather", engine)
weather_df.head()

2022-03-23 13:39:29,205 INFO sqlalchemy.engine.Engine SELECT dt, temperature, main FROM dbikes.real_time_weather
2022-03-23 13:39:29,207 INFO sqlalchemy.engine.Engine [raw sql] {}


,dt,temperature,main
0,1644922267,280,Clouds
1,1644922316,280,Clouds
2,1644922617,280,Clouds
3,1644922917,280,Clouds
4,1644923217,280,Clouds


In [11]:
#Drop duplicates
weather_df.drop_duplicates(inplace=True)

In [12]:
print(weather_df.shape)

(10392, 3)


In [13]:
#Changes the name of the column
availability_df.rename(columns={"last_update": "dt"}, inplace=True)
availability_df.columns

Index(['number', 'available_bikes', 'available_stands', 'dt'], dtype='object')

In [20]:
availability_df["dt"] = availability_df["dt"].astype(np.int64)
availability_df.dtypes

number              int64
available_bikes     int64
available_stands    int64
dt                  int64
dtype: object

In [21]:
weather_df["dt"]= weather_df["dt"].astype(np.int64)
weather_df.dtypes

dt              int64
temperature     int64
main           object
dtype: object

In [23]:
availability_df.sort_values(by="dt", inplace=True)
weather_df.sort_values(by="dt", inplace=True)

In [24]:
df=pd.merge_asof(availability_df, weather_df, on="dt")
df.head()

,number,available_bikes,available_stands,dt,temperature,main
0,86,0,38,1644921498000,289,Clouds
1,55,14,21,1644921501000,289,Clouds
2,80,22,18,1644921515000,289,Clouds
3,32,15,15,1644921522000,289,Clouds
4,99,15,15,1644921535000,289,Clouds


In [25]:
df.shape

(717692, 6)